In [ ]:
!pip install --no-deps /kaggle/input/map-utilities/bitsandbytes-0.47.0-py3-none-manylinux_2_24_x86_64.whl

In [ ]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

ROOT_PATH = os.getcwd()
if "/kaggle" in ROOT_PATH:
    ROOT_PATH = "/kaggle/input"
    sys.path.append(os.path.join(ROOT_PATH, "map-utilities"))

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

import torch
from datasets import Dataset
from peft import PeftModel
from transformers import BitsAndBytesConfig

from utils import (
    stringify_input,
    get_model_name,
    get_sequence_classifier,
    get_tokenizer,
    get_training_arguments,
    get_trainer,
)

In [ ]:
le = joblib.load(os.path.join(ROOT_PATH, "map-utilities", "label_encoder.joblib"))
n_classes = len(le.classes_)

In [ ]:
USE_LORA = True
USE_QLORA = False
MAX_LEN = 256

MODEL_VARIATIONS = [
    {
        "model_name": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "qwen-3/transformers/14b/1"
        ),
        "adapter_path": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "qwen3-14b-qlora-4bit/transformers/default/1"
        ),
        "submission_file": "submission_qwen3_14b.csv"
    },
    {
        "model_name": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "gemma-2/transformers/gemma-2-9b-it/2"
        ),
        "adapter_path": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "gemma2-9b-it-qlora-4bit/transformers/default/1"
        ),
        "submission_file": "submission_gemma2_9b.csv"
    },
    {
        "model_name": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "deepseek-r1-0528/transformers/deepseek-r1-0528-qwen3-8b/1"
        ),
        "adapter_path": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "deepseek-r1-0528-qwen3-8b-qlora-4bit/transformers/default/1"
        ),
        "submission_file": "submission_deepseek_r1_0528_qwen3_8b.csv"
    },
    {
        "model_name": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "deepseek-math/pytorch/deepseek-math-7b-instruct/1"
        ),
        "adapter_path": get_model_name(
            "/kaggle" in ROOT_PATH, ROOT_PATH, "deepseek-math-7b-instruct-qlora-4bit/transformers/default/1"
        ),
        "submission_file": "submission_deepseek_math_7b.csv"
    },
]

In [ ]:
TRAIN_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "train.csv")
TEST_PATH = os.path.join(ROOT_PATH, "map-charting-student-math-misunderstandings", "test.csv")

In [ ]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

In [ ]:
print("Training Shape:", train_df.shape)
print("Testing Shape:", test_df.shape)

In [ ]:
idx = train_df.Category.str.contains("True", case=False)
tmp = train_df.loc[idx].copy()
tmp["c"] = tmp.groupby(["QuestionId", "MC_Answer"]).MC_Answer.transform("count")
tmp = tmp.sort_values("c", ascending=False)
tmp = tmp.drop_duplicates(["QuestionId"])
tmp = tmp[["QuestionId", "MC_Answer"]]
tmp["is_mc_answer_correct"] = True

train_df = train_df.merge(tmp, on=["QuestionId", "MC_Answer"], how="left")
train_df.is_mc_answer_correct = train_df.is_mc_answer_correct.fillna(False)

test_df = test_df.merge(tmp, on=["QuestionId", "MC_Answer"], how="left")
test_df.is_mc_answer_correct = test_df.is_mc_answer_correct.fillna(False)

In [ ]:
qlora_config = {
    "torch_dtype": torch.float16,
    "device_map": "auto"
}
if USE_QLORA:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_storage=torch.float16,
    )
    qlora_config["quantization_config"] = bnb_config

In [ ]:
def predict_test_data(model_name, adapter_path, submission_file):
    seq_model = get_sequence_classifier(model_name, n_classes, qlora_config)
    tokenizer = get_tokenizer(model_name)

    def tokenize_function(examples):
        return tokenizer(examples["stringified_input"], truncation=True)

    if (
        "gemma" in model_name.lower()
        or "qwen" in model_name.lower()
        or "deepseek-math" in model_name.lower()
    ):
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.eos_token_id
        seq_model.config.pad_token_id = tokenizer.pad_token_id

    if USE_LORA:
        seq_model = PeftModel.from_pretrained(seq_model, adapter_path)

    test_df["stringified_input"] = test_df.apply(
        lambda row: stringify_input(row, model_name), axis=1
    )

    test_ds = Dataset.from_pandas(test_df[["stringified_input"]])
    test_ds = test_ds.map(tokenize_function, batched=True)

    training_args = get_training_arguments(bf16_support="/kaggle" not in ROOT_PATH)
    trainer = get_trainer(
        seq_model,
        tokenizer,
        training_args,
        test_ds,
        test_ds,
    )

    predictions = trainer.predict(test_ds)
    probs = torch.nn.functional.softmax(
        torch.tensor(predictions.predictions), dim=1
    ).numpy()

    top3 = np.argsort(-probs, axis=1)[:, :3]

    flat_top3 = top3.flatten()
    decoded_labels = le.inverse_transform(flat_top3)
    top3_labels = decoded_labels.reshape(top3.shape)

    joined_preds = ["|".join(row) for row in top3_labels]

    sub = pd.DataFrame(
        {"row_id": test_df.row_id.values, "Category:Misconception": joined_preds}
    )
    sub.to_csv(submission_file, index=False)

    del seq_model
    del tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
for model_variation in MODEL_VARIATIONS:
    predict_test_data(
        model_name=model_variation["model_name"],
        adapter_path=model_variation["adapter_path"],
        submission_file=model_variation["submission_file"]
    )

### Ensemble Predictions

In [ ]:
from collections import defaultdict


# https://www.kaggle.com/code/bibanh/lb-0-944-the-art-of-ensemble#4.-ENSEMBLE-EVERYTHING
def get_top_k_ensemble(list_of_predictions, k=3):
    predictions = []
    weights = []
    for lp in list_of_predictions:
        predictions.append(lp.split("|"))
        weights.append(4)
    score = defaultdict(int)

    for i, lst in enumerate(predictions):
        weight = weights[i]
        for rank, item in enumerate(lst):
            score[item] += (len(lst) - rank) * weight

    sorted_items = sorted(score.items(), key=lambda x: -x[1])
    return ' '.join([item for item, _ in sorted_items[:k]])

In [ ]:
dfs = []
for model_variation in MODEL_VARIATIONS:
    df = pd.read_csv(model_variation["submission_file"])
    dfs.append(df)

ensemble_df = dfs[0][['row_id']].copy()
for i, df in enumerate(dfs):
    model_name = MODEL_VARIATIONS[i]["submission_file"].replace("submission_", "").replace(".csv", "")
    ensemble_df[f"predictions_{model_name}"] = df["Category:Misconception"]

print("Ensemble df shape:", ensemble_df.shape)

In [ ]:
ensemble_df["Category:Misconception"] = ensemble_df.apply(
    lambda row: get_top_k_ensemble(
        [row[f"predictions_{model_name}"] for model_name in MODEL_VARIATIONS],
    ),
    axis=1,
)
ensemble_df[['row_id', 'Category:Misconception']].to_csv('submission.csv', index = False)
pd.read_csv('submission.csv')